In [1]:
%pip install scvi

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import scanpy as sc
import scvi
import pandas as pd

# 读取基因表达数据
gene_expression_data = pd.read_csv('data/scRNA_seq_for_hw3_hw5.tsv', index_col=0)

# 读取标签数据
labels = pd.read_csv('data/label_for_hw3_hw5.tsv', index_col=0)

# 创建AnnData对象
adata = sc.AnnData(X=gene_expression_data.T)  
adata.obs['labels'] = labels['label']  

# 规范化数据
sc.pp.normalize_total(adata, target_sum=1e4)

# 对数化数据
sc.pp.log1p(adata)

# 准备数据，使用scvi
scvi.external.pp.scvi_prepare(adata, 'labels')

# 训练VAE模型
vae = scvi.model.SCVI(adata)
vae.train()

# 获取降维表示
adata.obsm["X_vae"] = vae.get_latents()

# 使用UMAP进行降维
sc.tl.umap(adata, key_added="umap_vae", min_dist=0.3)

# 可视化
sc.pl.umap(adata, color='labels', palette='viridis', save='umap_vae_plot.png')
